# Prueba Tecnica LT Challenge

En este notebook, se enfocará en los dos enfoques diferentes para cada una de las funciones, tiempo de ejecución y optimización de memoria. Tomaremos q2 como base de nuestra evaluación debido a que la linea logica es similar y se repite en todas las funciones. Asi como las primeras versiones del codigo la tomaremos como benchmark para evaluar resultados. Ademas, anadiremos el siguiente banco de ideas donde disponeremos de conceptos o tecnologias que podamos incluir o evaluar hasta la entrega de este proyecto:

| Foco            | Concepto                                                         | Estado                           |
|-----------------|------------------------------------------------------------------|----------------------------------|
| tiempo & memoria| **Pandas Basico**: Libreria estandar para procesamiento de datos | Incluida 1a version              |
| tiempo & memoria| **Chunks**: Es necesario particionar los datos                   | Incluida 1a version              |
| tiempo          | **Multiprocessing**: Procesamiento paralelo                      | Pendiente                        |
| tiempo          | **Polars**: Libreria escrita en Rust                             | Pendiende *Valdria la pena evaluarlo* |
| memoria         | **Generators**: Lazy evaluation e iterators sirven para optimizacion de memoria | Pendiente                        |
| memoria         | **Pandas Avanzado**: Aplicar tecnicas como sparse                | pendiente                        |


----
## Análisis top 10 de los emojis más usados en los Tweets

### Preprocesamiento e Importación de Librerías

In [13]:
import pandas as pd
import emoji
import time
import os
from typing import List, Tuple
from memory_profiler import memory_usage
from memory_profiler import profile

In [3]:
from q2_memory import q2_memory
from stats import get_file_path

# Usaremos los datos proveídos en el challenge pero desde google cloud storage
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../sa.json"
bucket_name = 'test_jobs144'
file_name = 'lt/farmers-protest-tweets-2021-2-4.json'
temp_file_path = get_file_path(bucket_name, file_name)

### Codigo Benchmark
Este código fue desarrollado de forma básica a los dos días de iniciar el challenge, con el fin de tener una base para trabajar. Esta función es igual para q2_memory y q2_time por practicidad; a lo largo del proyecto, esto cambiará.


In [14]:
%load_ext memory_profiler

@profile
def q2(file_path: str) -> List[Tuple[str, int]]:
    """
    Analyze Twitter data to count emojis for the top 10 emojis.
    
    Args:
    file_path (str): Path to file containing Twitter data.
    
    Returns:
    List[Tuple[str, int]]: Top 10 days with most emojis and their emoji counts
    """
    
    chunks = pd.read_json(file_path, lines=True, chunksize=10000) # Particionamos en chunks de 10000 lineas cada uno
    
    dfs = []
    
    for chunk in chunks: # Por cada chuck extraemos un dataframe con los emojis y sus contadores
        emojis_list = chunk['content'].apply(extract_emojis).explode() # Extraemos series.pandas continiendo lista de emojis para luego aplanarlos en una serie de strings 
        emojis_count = emojis_list.to_frame(name='emoji').groupby('emoji').size().reset_index(name='count') # Realizamos conteo por emoji
        dfs.append(emojis_count)
    
    all_counts = pd.concat(dfs) # Concatenamos dataframes derivados de los chunks
    final_counts = all_counts.groupby('emoji')['count'].sum().reset_index() # Realizamos la correspondiente agregacion de acuerdo al emoji
    top_10_days = final_counts.sort_values('count', ascending=False).head(10) # Seleccionamos el top 10
    
    return [(row['emoji'], int(row['count'])) for _, row in top_10_days.iterrows()]

def extract_emojis(text: str) -> list: # Extrae todos los emojis que pueda tener un tweet
    """Get emojis in a given text."""
    return [x for x in text if x in emoji.EMOJI_DATA]

#### Extraemos metricas de uso memoria y tiempo de procesamiento Benchmark

In [15]:
%mprun -f q2 q2(temp_file_path)

ERROR: Could not find file /tmp/ipykernel_1599/4263676936.py



Filename: /usr/local/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185    279.9 MiB    279.9 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187    279.9 MiB      0.0 MiB           1                   prof = get_prof()
  1188    338.8 MiB     58.9 MiB           1                   val = prof(func)(*args, **kwargs)
  1189    338.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190    338.8 MiB      0.0 MiB           1                   return val

In [16]:
mem_benchmark_usage = memory_usage((q2, (temp_file_path,), {}))

ERROR: Could not find file /tmp/ipykernel_1599/4263676936.py


In [17]:
print(f"Average memory usage: {sum(mem_benchmark_usage) / len(mem_benchmark_usage)} MiB")
print(f"Maximum memory usage: {max(mem_benchmark_usage)} MiB")

Average memory usage: 427.6349664135514 MiB
Maximum memory usage: 471.4140625 MiB


In [7]:
import time
start_time = time.time()
exam = q1_memory(file_path)
end_time = time.time()
total_time = end_time - start_time
print(total_time)
print(end_time)

5.8084704875946045
1727450197.4192607


In [4]:
exam = q2_memory(test_file_path)
exam

[('🤫', 2), ('🤔', 2), ('🌾', 1), ('🚜', 1), ('💪', 1)]

In [5]:
exam = q3_memory(test_file_path)
exam

[('mandeeppunia1', 2),
 ('DelhiPolice', 1),
 ('Kisanektamorcha', 1),
 ('ReallySwara', 1),
 ('narendramodi', 1),
 ('rohini_sgh', 1)]